In [1]:
from pathlib import Path

import numpy as np
import pandas as pd

In [2]:
from IPython.core.interactiveshell import InteractiveShell
# 显示所有输出
InteractiveShell.ast_node_interactivity = 'all'

In [3]:
csv = pd.read_csv('./data/Spaceship Titanic/train.csv')
csv.shape

(8693, 14)

In [4]:
# 查看前几行
csv.head(3)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False


# 清理空值

In [5]:
# 存在大量空值
# 题上声称“从事故中恢复的残缺数据”
pd.isna(csv).sum().sort_values(ascending=False)

CryoSleep       217
ShoppingMall    208
VIP             203
HomePlanet      201
Name            200
Cabin           199
VRDeck          188
FoodCourt       183
Spa             183
Destination     182
RoomService     181
Age             179
PassengerId       0
Transported       0
dtype: int64

In [6]:
# 填充0值
csv.loc[:, ['Age', 'RoomService', 'FoodCourt',
            'ShoppingMall', 'Spa', 'VRDeck']] = csv.loc[:, ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].fillna(0)
# 填充False值
csv.loc[:, ['VIP', 'CryoSleep']] = csv.loc[:,
                                           ['VIP', 'CryoSleep']].fillna(False)
# 填充空字符串
csv[['HomePlanet', 'Destination']] = csv[[
    'HomePlanet', 'Destination']].fillna('')

# 船舱信息

In [7]:
# 未知的船舱号用-1代替
csv['Cabin'] = csv['Cabin'].fillna('/-1/')

In [8]:
# 参数expand=True表示返回表单
csv[['Deck', 'Cabin_num', 'Side']
    ] = csv['Cabin'].str.split('/', expand=True)

# 组团信息

In [9]:
csv[['GroupID', 'MemberID']] = csv['PassengerId'].str.split('_', expand=True)

# 转化数据类型

In [10]:
# 浮点数转整数
csv['Age'] = csv['Age'].astype(int)
# 布尔值转整数0-1
csv[['CryoSleep', 'VIP', 'Transported']] = csv[[
    'CryoSleep', 'VIP', 'Transported']].astype(int)
# 字符串转整数
csv[['GroupID', 'MemberID', 'Cabin_num']] = csv[[
    'GroupID', 'MemberID', 'Cabin_num']].astype(int)

# 出发地/目的地对应

In [11]:
def get_map(csv, col_name):
    result = {}
    for idx, key in enumerate(csv[col_name].value_counts().index):
        result[key] = idx
    return result

In [12]:
# 统计所有出现过的值，创建字符串到整数的映射
HomePlanet_map = get_map(csv, 'HomePlanet')
Destination_map = get_map(csv, 'Destination')
Deck_map = get_map(csv, 'Deck')
Side_map = get_map(csv, 'Side')

HomePlanet_map
Destination_map
Deck_map
Side_map

{'Earth': 0, 'Europa': 1, 'Mars': 2, '': 3}

{'TRAPPIST-1e': 0, '55 Cancri e': 1, 'PSO J318.5-22': 2, '': 3}

{'F': 0, 'G': 1, 'E': 2, 'B': 3, 'C': 4, 'D': 5, 'A': 6, '': 7, 'T': 8}

{'S': 0, 'P': 1, '': 2}

In [13]:
# 将字符串转化为对应的整数值
csv['HomePlanet'] = csv['HomePlanet'].apply(
    lambda x: HomePlanet_map.get(x)).astype(int)
csv['Destination'] = csv['Destination'].apply(
    lambda x: Destination_map.get(x)).astype(int)
csv['Deck'] = csv['Deck'].apply(lambda x: Deck_map.get(x)).astype(int)
csv['Side'] = csv['Side'].apply(lambda x: Side_map.get(x)).astype(int)

# 结果

In [14]:
# 调整列顺序
# 忽略无用的列['PassengerId', 'Cabin', 'Name']
csv = csv[[
    # 基本信息
    'GroupID', 'MemberID', 'HomePlanet', 'Destination',
    # 个人信息
    'CryoSleep', 'VIP', 'Age',
    # 消费
    'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
    # 船舱
    'Deck', 'Cabin_num', 'Side',
    'Transported']]

In [15]:
# 清理空值的结果
pd.isna(csv).sum().sort_values(ascending=False)

GroupID         0
MemberID        0
HomePlanet      0
Destination     0
CryoSleep       0
VIP             0
Age             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Deck            0
Cabin_num       0
Side            0
Transported     0
dtype: int64

In [16]:
csv.head(3)

,GroupID,MemberID,HomePlanet,Destination,CryoSleep,VIP,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Deck,Cabin_num,Side,Transported
0,1,1,1,0,0,0,39,0.0,0.0,0.0,0.0,0.0,3,0,1,0
1,2,1,0,0,0,0,24,109.0,9.0,25.0,549.0,44.0,0,0,0,1
2,3,1,1,0,0,1,58,43.0,3576.0,0.0,6715.0,49.0,6,0,0,0


In [17]:
# 保存为CSV
csv.to_csv('./data/train_clear.csv')
# 保存为Pickle
csv.to_pickle('./data/train_clear.pickle')